In [1]:
import pandas as pd
import re

In [2]:
raw_df = pd.read_csv("Manually_Categorized.csv")
raw2_df = pd.read_csv("workbook.csv")

In [3]:
df = raw_df[["Project Name", "Start Date", "Amount Awarded", "Category"]].rename(columns = dict(Category = "cat0"))

In [4]:
df2 = raw2_df[["Project Name", "Start Date", "Amount Awarded"]]

In [5]:
#df = pd.concat([df1,df2], sort = False).drop_duplicates().drop(["Project Name"], axis = 1)

In [6]:
#df = df1.merge(df2, on = ["Project Name","Start Date", "Amount Awarded"])

In [7]:
keywords = dict(Medical = [], Food = [] ,Others = [])

In [8]:
def convert_to_non_special_cw_list(_str):
    return re.sub(r"[^A-Za-z0-9\s\-]", "", _str.lower()).split(" ")

In [9]:
def update_keywords(cycle):
    #extract keywords
    for cat in keywords.keys():
        cat_string = " ".join(df[df[f"cat{cycle}"] == cat]["Project Name"].str.lower())
        keywords[cat] = list(set(convert_to_non_special_cw_list(cat_string)))
        
    #remove intersection keywords
    temp_keywords = dict(keywords)
    for cat in keywords.keys():
        other_cats = list(set(keywords.keys()) - set([cat]))
        final_cat_keywords = set(temp_keywords[cat]) - set(temp_keywords[other_cats[0]]) - set(temp_keywords[other_cats[1]])
        keywords[cat] = list(final_cat_keywords)

In [10]:
def categorizer(name):
    name_list = convert_to_non_special_cw_list(name.lower())
    
    for cat in keywords.keys():
        if set(name_list).intersection(set(keywords[cat])):
            return cat
    else:
        if ("medical equipment" in name.lower()) or ("medical suppl" in name.lower()) \
        or ("hospital equipment" in name.lower()) or ("hospital suppl" in name.lower()):
            return "Medical"
        elif ("equipment" in name.lower()):
            return "Others"
        return "Not classified"

In [11]:
def percentage_calcu(category): 
    if category == "Not classified": 
        cat_total_amount_awarded = df[df.iloc[:,-2] == "Not classified"]["Amount Awarded"].sum()
    else: 
        cat_total_amount_awarded = df[df.iloc[:,-1] == category]["Amount Awarded"].sum()
    percent_budget_not_categorized = cat_total_amount_awarded/df["Amount Awarded"].sum() * 100
    return percent_budget_not_categorized
        

In [12]:
cycle = 0
previous_unknown_count = 0
while True:
    update_keywords(cycle)
    cycle += 1
    df[f"cat{cycle}"] = df[f"cat{cycle-1}"]
    df.loc[df[f"cat{cycle}"]=="Not classified", f"cat{cycle}"] = df.loc[df[f"cat{cycle}"]=="Not classified", "Project Name"].map(categorizer)
    current_unknown_count = (df[f"cat{cycle}"] == "Not classified").sum()
    print(current_unknown_count)
    if current_unknown_count == previous_unknown_count:
        break
    else:
        previous_unknown_count = current_unknown_count

2001
1300
1265
1262
1262


In [13]:
df2.head()

,Project Name,Start Date,Amount Awarded
0,Human Grade Disinfectant Solution and Face Mas...,2020-04-08,108864.0
1,Procurement of relief goods,2020-04-16,632800.0
2,Procurement of relief goods,2020-04-16,632800.0
3,Procurement of Relief Goods,2020-04-17,1535500.0
4,Procurement of Various Medical Supplies for It...,2020-03-24,64790000.0


In [14]:
df2["Category"] = "Not classified"

C:\Users\Joyce Heidi Ong\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df2.Category = df2.loc[:,"Project Name"].map(categorizer)

C:\Users\Joyce Heidi Ong\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
keywords.keys()

dict_keys(['Medical', 'Food', 'Others'])

In [17]:
df2.Category.unique()

array(['Medical', 'Not classified', 'Others', 'Food'], dtype=object)

In [18]:
df["Final Categorization"] = df.iloc[:,-1].replace("Not classified", "Others")

In [19]:
#list(map(percentage_calcu,["Not classified","Medical", "Food", "Others"]))

[1.6181937276321727, 64.99204259795222, 28.49387135145384, 6.514086050593932]

In [20]:
df2.to_csv("auto-classified.csv")